<a href="https://colab.research.google.com/github/Alihassan7726/LSTM-Bi-LSTM/blob/main/BBC_News_headlines_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, LSTM, Dropout, Activation , GlobalMaxPooling1D , Embedding
from tensorflow.keras.callbacks import *
import pandas as pd

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
pip install tqdm

In [ ]:
pd.set_option('display.width',1000)
pd.set_option('max_colwidth', 1000) 
import tqdm
from tqdm import tqdm
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stpwrds = set(stopwords.words('english'))
stpwrds.discard('not')
stpwrds.discard('no')
import os
import re
from nltk.stem import WordNetLemmatizer as wnl
import sklearn


### Getting the Data

In [ ]:
# Data is fetched and stored in tmp folder with name bbc-text.csv
!wget --no-check-certificate \
    https://storage.googleapis.com/dataset-uploader/bbc/bbc-text.csv \
    -O /tmp/bbc-text.csv

--2021-04-27 18:07:47--  https://storage.googleapis.com/dataset-uploader/bbc/bbc-text.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.7.208, 172.217.15.112, 172.217.12.240, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.7.208|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5057493 (4.8M) [text/csv]
Saving to: ‘/tmp/bbc-text.csv’

/tmp/bbc-text.csv   100%[===================>]   4.82M  --.-KB/s    in 0.02s   

2021-04-27 18:07:47 (215 MB/s) - ‘/tmp/bbc-text.csv’ saved [5057493/5057493]



In [ ]:
df = pd.read_csv('/tmp/bbc-text.csv' , delimiter = ',')
df.head()

,category,text
0,tech,tv future in the hands of viewers with home theatre systems plasma high-definition tvs and digital video recorders moving into the living room the way people watch tv will be radically different in five years time. that is according to an expert panel which gathered at the annual consumer electronics show in las vegas to discuss how these new technologies will impact one of our favourite pastimes. with the us leading the trend programmes and other content will be delivered to viewers via home networks through cable satellite telecoms companies and broadband service providers to front rooms and portable devices. one of the most talked-about technologies of ces has been digital and personal video recorders (dvr and pvr). these set-top boxes like the us s tivo and the uk s sky+ system allow people to record store play pause and forward wind tv programmes when they want. essentially the technology allows for much more personalised tv. they are also being built-in to h...
1,business,worldcom boss left books alone former worldcom boss bernie ebbers who is accused of overseeing an $11bn (£5.8bn) fraud never made accounting decisions a witness has told jurors. david myers made the comments under questioning by defence lawyers who have been arguing that mr ebbers was not responsible for worldcom s problems. the phone company collapsed in 2002 and prosecutors claim that losses were hidden to protect the firm s shares. mr myers has already pleaded guilty to fraud and is assisting prosecutors. on monday defence lawyer reid weingarten tried to distance his client from the allegations. during cross examination he asked mr myers if he ever knew mr ebbers make an accounting decision . not that i am aware of mr myers replied. did you ever know mr ebbers to make an accounting entry into worldcom books mr weingarten pressed. no replied the witness. mr myers has admitted that he ordered false accounting entries at the request of former worldcom chief fina...
2,sport,tigers wary of farrell gamble leicester say they will not be rushed into making a bid for andy farrell should the great britain rugby league captain decide to switch codes. we and anybody else involved in the process are still some way away from going to the next stage tigers boss john wells told bbc radio leicester. at the moment there are still a lot of unknowns about andy farrell not least his medical situation. whoever does take him on is going to take a big big gamble. farrell who has had persistent knee problems had an operation on his knee five weeks ago and is expected to be out for another three months. leicester and saracens are believed to head the list of rugby union clubs interested in signing farrell if he decides to move to the 15-man game. if he does move across to union wells believes he would better off playing in the backs at least initially. i m sure he could make the step between league and union by being involved in the centre said wells. ...
3,sport,yeading face newcastle in fa cup premiership side newcastle united face a trip to ryman premier league leaders yeading in the fa cup third round. the game - arguably the highlight of the draw - is a potential money-spinner for non-league yeading who beat slough in the second round. conference side exeter city who knocked out doncaster on saturday will travel to old trafford to meet holders manchester united in january. arsenal were drawn at home to stoke and chelsea will play host to scunthorpe. the only other non-league side in the draw are hinckley united who held brentford to a goalless draw on sunday. they will meet league one leaders luton if they win their replay against martin allen s team at griffin park. a number of premiership teams face difficult away games against championship sides on the weekend of 8/9 january. third-placed everton visit plymouth liverpool travel to burnley crystal palace go to sunderland fulham face carling cup semi-finalists watford bolton...
4,entertainment,ocea

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2225 entries, 0 to 2224
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  2225 non-null   object
 1   text      2225 non-null   object
dtypes: object(2)
memory usage: 34.9+ KB


In [ ]:
df['category'].value_counts()

sport            511
business         510
politics         417
tech             401
entertainment    386
Name: category, dtype: int64

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
def clean_text(text):  
    lower = text.lower()
    words = re.sub(r"(@[A-Za-z]+)|([^A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", lower )  
    #pattren = r"[^\w\s\d]"
    #words = re.sub(pattren, "", lower)
    words2 = words.split()
    #print(words2)
    final_words =  [wnl().lemmatize(word , pos = 'v') for word in words2 if word not in stopwords.words('english')]
    final_words = ' '.join(final_words)
    return(final_words)
    
df['Cleaned-text'] = df['text'].apply(clean_text)
df.head()

,category,text,Cleaned-text
0,tech,tv future in the hands of viewers with home theatre systems plasma high-definition tvs and digital video recorders moving into the living room the way people watch tv will be radically different in five years time. that is according to an expert panel which gathered at the annual consumer electronics show in las vegas to discuss how these new technologies will impact one of our favourite pastimes. with the us leading the trend programmes and other content will be delivered to viewers via home networks through cable satellite telecoms companies and broadband service providers to front rooms and portable devices. one of the most talked-about technologies of ces has been digital and personal video recorders (dvr and pvr). these set-top boxes like the us s tivo and the uk s sky+ system allow people to record store play pause and forward wind tv programmes when they want. essentially the technology allows for much more personalised tv. they are also being built-in to h...,tv future hand viewers home theatre systems plasma highdefinition tvs digital video recorders move live room way people watch tv radically different five years time accord expert panel gather annual consumer electronics show las vegas discuss new technologies impact one favourite pastimes us lead trend program content deliver viewers via home network cable satellite telecoms company broadband service providers front room portable devices one talkedabout technologies ces digital personal video recorders dvr pvr settop box like us tivo uk sky system allow people record store play pause forward wind tv program want essentially technology allow much personalise tv also builtin highdefinition tv set big business japan us slower take europe lack highdefinition program people forward wind advert also forget abide network channel schedule put together alacarte entertainment us network cable satellite company worry mean term advertise revenues well brand identity viewer loyalty channel alth...
1,business,worldcom boss left books alone former worldcom boss bernie ebbers who is accused of overseeing an $11bn (£5.8bn) fraud never made accounting decisions a witness has told jurors. david myers made the comments under questioning by defence lawyers who have been arguing that mr ebbers was not responsible for worldcom s problems. the phone company collapsed in 2002 and prosecutors claim that losses were hidden to protect the firm s shares. mr myers has already pleaded guilty to fraud and is assisting prosecutors. on monday defence lawyer reid weingarten tried to distance his client from the allegations. during cross examination he asked mr myers if he ever knew mr ebbers make an accounting decision . not that i am aware of mr myers replied. did you ever know mr ebbers to make an accounting entry into worldcom books mr weingarten pressed. no replied the witness. mr myers has admitted that he ordered false accounting entries at the request of former worldcom chief fina...,worldcom boss leave book alone former worldcom boss bernie ebbers accuse oversee bn bn fraud never make account decisions witness tell jurors david myers make comment question defence lawyers argue mr ebbers responsible worldcom problems phone company collapse prosecutors claim losses hide protect firm share mr myers already plead guilty fraud assist prosecutors monday defence lawyer reid weingarten try distance client allegations cross examination ask mr myers ever know mr ebbers make account decision aware mr myers reply ever know mr ebbers make account entry worldcom book mr weingarten press reply witness mr myers admit order false account entries request former worldcom chief financial officer scott sullivan defence lawyers try paint mr sullivan admit fraud testify later trial mastermind behind worldcom account house card mr ebbers team meanwhile look portray affable boss admission pe graduate economist whatever abilities mr ebbers transform worldcom relative unknown bn tel

In [ ]:
X = df['Cleaned-text']
y = df['category']


In [ ]:
import sklearn
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size = 0.1,random_state = 42,shuffle= True,
                                                      stratify = df['category'])
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
print(y_test.value_counts())

(2002,)
(223,)
(2002,)
(223,)
business         51
sport            51
politics         42
tech             40
entertainment    39
Name: category, dtype: int64


In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
def create_corpus(df):
    corpus=[]
    for text in tqdm(df):
        words=[word for word in word_tokenize(text)]
        corpus.append(words)
    return corpus

In [ ]:
train_corpus = create_corpus(X_train)
test_corpus = create_corpus(X_test)
test_corpus[:1]

100%|██████████| 223/223 [00:00<00:00, 1237.90it/s]


[['bridge',
  'could',
  'miss',
  'rest',
  'season',
  'chelsea',
  'leftback',
  'wayne',
  'bridge',
  'could',
  'miss',
  'rest',
  'season',
  'suspect',
  'break',
  'ankle',
  'england',
  'international',
  'hurt',
  'innocuous',
  'challenge',
  'alan',
  'shearer',
  'londoners',
  'fa',
  'cup',
  'defeat',
  'newcastle',
  'sunday',
  'think',
  'big',
  'injury',
  'say',
  'jose',
  'mourinho',
  'whose',
  'chelsea',
  'team',
  'week',
  'meet',
  'barcelona',
  'champion',
  'league',
  'liverpool',
  'carling',
  'cup',
  'final',
  'william',
  'gallas',
  'damien',
  'duff',
  'could',
  'also',
  'miss',
  'barcelona',
  'game',
  'france',
  'defender',
  'gallas',
  'ireland',
  'winger',
  'duff',
  'pick',
  'injuries',
  'close',
  'stag',
  'chelsea',
  'defeat',
  'st',
  'jam',
  'park',
  'bridge',
  'go',
  'mourinho',
  'use',
  'three',
  'substitute',
  'knock',
  'plus',
  'carlo',
  'cudicini',
  'red',
  'card',
  'mean',
  'chelsea',
  'finish',


In [ ]:
oov_tok = '<OOV>' #  ‘<OOV>’ that we put if the word is not listed in the dictionary.
tokenizer_obj = Tokenizer(num_words = 10000, # max number of words to be taken
                         filters =  '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t0-9',
                          oov_token = oov_tok)

tokenizer_obj.fit_on_texts(train_corpus) # methods `texts_to_sequences` or `texts_to_matrix`.
train_sequences=tokenizer_obj.texts_to_sequences(train_corpus) # Transforms each text in texts to a sequence of integers.
test_sequences=tokenizer_obj.texts_to_sequences(test_corpus)

test_sequences[:1]

[[2072,
  18,
  366,
  914,
  316,
  437,
  1,
  3188,
  2072,
  18,
  366,
  914,
  316,
  1034,
  221,
  2788,
  77,
  175,
  1821,
  1,
  455,
  897,
  4890,
  8094,
  1356,
  273,
  743,
  980,
  379,
  36,
  181,
  474,
  2,
  1565,
  1337,
  845,
  437,
  155,
  102,
  91,
  1943,
  303,
  490,
  599,
  3206,
  273,
  236,
  1676,
  3157,
  3339,
  3340,
  18,
  6,
  366,
  1943,
  14,
  210,
  2200,
  3157,
  239,
  3193,
  3340,
  617,
  2066,
  242,
  1623,
  437,
  743,
  1414,
  831,
  877,
  2072,
  13,
  1337,
  16,
  55,
  4311,
  1501,
  2018,
  9402,
  5934,
  1138,
  341,
  126,
  437,
  651,
  14,
  726,
  1,
  124,
  5934,
  4,
  22,
  1943,
  506,
  14,
  3206,
  273,
  236,
  263,
  366,
  1478,
  1,
  2720,
  442,
  43,
  40,
  8,
  196,
  739,
  358,
  1723,
  1943,
  104,
  528,
  1887,
  680,
  2072,
  1,
  1,
  9325,
  148,
  160,
  270,
  7582,
  423,
  980,
  263,
  3157,
  1,
  1,
  834,
  965,
  18,
  8816,
  7859,
  1700,
  9974,
  1487,
  1,
  9998,
  86

In [ ]:
word_index=tokenizer_obj.word_index
lenght = len(word_index)+1
print('Number of unique words:',lenght)

Number of unique words: 23938


In [ ]:
train_pad=pad_sequences(train_sequences,maxlen=300,truncating='post',padding='pre') # lists to array form
test_pad=pad_sequences(test_sequences,maxlen=300,truncating='post',padding='pre')

print(len(train_corpus))
print(train_pad.shape)
train_pad

2002
(2002, 300)


array([[   0,    0,    0, ..., 6110, 1655,  575],
       [   0,    0,    0, ...,  132, 1391,  160],
       [   0,    0,    0, ...,   11,    9,  252],
       ...,
       [   0,    0,    0, ...,  204, 2054,  860],
       [   0,    0,    0, ..., 8214, 6529, 7243],
       [ 906, 1011, 1137, ...,  492,    6, 1038]], dtype=int32)

In [ ]:
labels = list(set(y_train))
#train_labels = set(y_train.unique())
#validation_labels = set(y_test.unique())
#validation_labels
labels

['entertainment', 'tech', 'politics', 'sport', 'business']

In [ ]:
#y_train = pd.get_dummies(y_train)
#y_test = pd.get_dummies(y_test)
#print(y_train.shape)
#arr = np.array(y_train)
#print(arr.shape)


In [ ]:
tokenizer_obj = Tokenizer()
tokenizer_obj.fit_on_texts(labels)
training_label_seq = np.array(tokenizer_obj.texts_to_sequences(y_train))
validation_label_seq = np.array(tokenizer_obj.texts_to_sequences(y_test))
print(len(validation_label_seq))
validation_label_seq

223


array([[4],
       [5],
       [3],
       [4],
       [2],
       [1],
       [5],
       [1],
       [5],
       [3],
       [4],
       [3],
       [3],
       [4],
       [4],
       [4],
       [4],
       [3],
       [3],
       [5],
       [1],
       [5],
       [1],
       [2],
       [5],
       [2],
       [2],
       [4],
       [3],
       [3],
       [3],
       [2],
       [5],
       [2],
       [4],
       [5],
       [4],
       [3],
       [1],
       [4],
       [1],
       [5],
       [5],
       [4],
       [3],
       [2],
       [4],
       [3],
       [1],
       [4],
       [4],
       [4],
       [2],
       [2],
       [5],
       [5],
       [5],
       [3],
       [2],
       [2],
       [1],
       [5],
       [3],
       [5],
       [5],
       [5],
       [2],
       [5],
       [4],
       [5],
       [5],
       [3],
       [1],
       [5],
       [1],
       [5],
       [4],
       [1],
       [3],
       [3],
       [3],
       [3],
       [1],
    

In [ ]:
training_label_seq

array([[4],
       [4],
       [1],
       ...,
       [2],
       [4],
       [1]])

In [ ]:
#validation_label_seq = np.array(validation_label_seq).reshape(223,)
#training_label_seq = np.array(training_label_seq).reshape(len(X_train),)

In [ ]:
tokenizer_obj.word_index

{'business': 5, 'entertainment': 1, 'politics': 3, 'sport': 4, 'tech': 2}

In [ ]:
model = Sequential()
DIM = 300
MAX_LEN = 300
## Embedding layer
model.add(Embedding(lenght , DIM , input_length = MAX_LEN , trainable=True))
#model.compile('adam' , 'mse')

## LSTM layer
model.add(LSTM(DIM,return_sequences=True))

#Global Maxpooling
model.add(GlobalMaxPooling1D())

#Dense Layer
model.add(Dense(256,activation='relu')) 
model.add(Dense(5,activation='softmax')) 

#Add loss function, metrics, optimizer
opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    
#Adding callbacks
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=3)  
#mc=ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', save_best_only=True,verbose=1)

#Model summary
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 300, 300)          7181400   
_________________________________________________________________
lstm_7 (LSTM)                (None, 300, 300)          721200    
_________________________________________________________________
global_max_pooling1d_4 (Glob (None, 300)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 256)               77056     
_________________________________________________________________
dense_12 (Dense)             (None, 5)                 1285      
Total params: 7,980,941
Trainable params: 7,980,941
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
train_encode = lb.fit_transform(y_train)
test_encode = lb.fit_transform(y_test)

#train_encode = np.array(train_encode).reshape(len(X_train),1)
#test_encode = np.array(test_encode).reshape(len(X_test),1)
test_encode[:10]

array([3, 0, 2, 3, 4, 1, 0, 1, 0, 2])

In [ ]:
lb.inverse_transform(test_encode[:10])

array(['sport', 'business', 'politics', 'sport', 'tech', 'entertainment',
       'business', 'entertainment', 'business', 'politics'], dtype=object)

In [ ]:
#training_label_seq

In [ ]:
history = model.fit(train_pad,train_encode,batch_size=25,epochs=10,
                    validation_data=(test_pad,test_encode),verbose=1,callbacks=[es],
                    )

Epoch 1/10
81/81 [==============================] - 10s 113ms/step - loss: 1.4676 - accuracy: 0.3663 - val_loss: 0.3287 - val_accuracy: 0.9103
Epoch 2/10
81/81 [==============================] - 8s 102ms/step - loss: 0.1597 - accuracy: 0.9662 - val_loss: 0.1022 - val_accuracy: 0.9776
Epoch 3/10
81/81 [==============================] - 8s 103ms/step - loss: 0.0212 - accuracy: 0.9951 - val_loss: 0.0740 - val_accuracy: 0.9821
Epoch 4/10
81/81 [==============================] - 9s 106ms/step - loss: 0.0063 - accuracy: 0.9991 - val_loss: 0.0786 - val_accuracy: 0.9731
Epoch 5/10
81/81 [==============================] - 8s 104ms/step - loss: 0.0012 - accuracy: 0.9997 - val_loss: 0.1032 - val_accuracy: 0.9776
Epoch 6/10
81/81 [==============================] - 8s 103ms/step - loss: 0.0091 - accuracy: 0.9964 - val_loss: 0.1176 - val_accuracy: 0.9821
Epoch 00006: early stopping


In [ ]:
accr = model.evaluate(test_pad,test_encode)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

7/7 [==============================] - 0s 21ms/step - loss: 0.1176 - accuracy: 0.9821
Test set
  Loss: 0.118
  Accuracy: 0.982


In [ ]:
model.predict(test_pad , batch_size = 25)

array([[4.1819920e-12, 4.0197660e-06, 7.4723442e-07, 9.9998093e-01,
        1.4285516e-05],
       [9.9784565e-01, 1.8563581e-03, 2.7195885e-05, 2.3163636e-06,
        2.6847544e-04],
       [2.6164986e-07, 2.8022190e-04, 9.9796593e-01, 1.1510779e-03,
        6.0242205e-04],
       ...,
       [4.6469238e-02, 9.5050967e-01, 3.5456233e-04, 2.6022946e-04,
        2.4062551e-03],
       [9.1232666e-09, 5.3579264e-08, 4.9392752e-09, 5.0863562e-08,
        9.9999988e-01],
       [2.8025213e-06, 3.1941352e-05, 9.9743515e-01, 1.6270067e-04,
        2.3674536e-03]], dtype=float32)

In [ ]:
txt = ["blair prepares to name poll date tony blair is likely to name 5 may as election day when parliament returns from its easter break  the bbc s political editor has learned.  andrew marr says mr blair will ask the queen on 4 or 5 april to dissolve parliament at the end of that week. mr blair has so far resisted calls for him to name the day but all parties have stepped up campaigning recently. downing street would not be drawn on the claim  saying election timing was a matter for the prime minister.  a number 10 spokeswoman would only say:  he will announce an election when he wants to announce an election.  the move will signal a frantic week at westminster as the government is likely to try to get key legislation through parliament. the government needs its finance bill  covering the budget plans  to be passed before the commons closes for business at the end of the session on 7 april.  but it will also seek to push through its serious and organised crime bill and id cards bill. mr marr said on wednesday s today programme:  there s almost nobody at a senior level inside the government or in parliament itself who doesn t expect the election to be called on 4 or 5 april.  as soon as the commons is back after the short easter recess  tony blair whips up to the palace  asks the queen to dissolve parliament ... and we re going.  the labour government officially has until june 2006 to hold general election  but in recent years governments have favoured four-year terms."]
tokenizer_obj.fit_on_texts(train_corpus) # methods `texts_to_sequences` or `texts_to_matrix`.
seq = tokenizer_obj.texts_to_sequences(txt)
print(seq)
padded = pad_sequences(seq, maxlen=300)
print(padded.shape)
pred = model.predict(padded)
print(pred)
actual = np.argmax(pred) # index of maximum value
print(actual)
print(labels)


[[93, 195, 632, 530, 430, 93, 247, 195, 65, 86, 166, 582, 9394, 220, 56, 421, 1401, 1106, 4967, 2, 93, 1735, 198, 1489, 822, 9115, 582, 80, 101, 2, 93, 251, 195, 166, 16484, 3853, 548, 784, 3, 623, 10869, 84, 86, 676, 249, 60, 45, 1290, 3, 1, 1735, 243, 86, 243, 86, 79, 1735, 1454, 6304, 101, 1924, 24, 247, 95, 14, 303, 1218, 582, 24, 566, 230, 367, 10869, 730, 179, 80, 1825, 822, 1735, 5, 461, 664, 971, 963, 230, 1097, 230, 2, 4967, 432, 455, 374, 385, 2454, 655, 258, 1282, 24, 582, 68, 86, 10869, 822, 804, 730, 40, 645, 9394, 19363, 430, 93, 3853, 2792, 1489, 9115, 582, 52, 24, 2309, 820, 102, 200, 86, 300, 31, 2215, 16484, 144, 9]]
(1, 300)
[[1.2797618e-04 1.7614417e-02 6.8676579e-03 1.2305053e-01 8.5233945e-01]]
4
['entertainment', 'tech', 'politics', 'sport', 'business']


In [ ]:
model.predict_classes(padded)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array([4])

In [ ]:
txt = ["tigers wary of farrell gamble leicester say they will not be rushed into making a bid for andy farrell should the great britain rugby league captain decide to switch codes. we and anybody else involved in the process are still some way away from going to the next stage tigers boss john wells told bbc radio leicester. at the moment there are still a lot of unknowns about andy farrell not least his medical situation. whoever does take him on is going to take a big big gamble. farrell who has had persistent knee problems had an operation on his knee five weeks ago and is expected to be out for another three months. leicester and saracens are believed to head the list of rugby union clubs interested in signing farrell if he decides to move to the 15-man game. if he does move across to union wells believes he would better off playing in the backs at least initially. i m sure he could make the step between league and union by being involved in the centre said wells."]
tokenizer_obj.fit_on_texts(train_corpus) # methods `texts_to_sequences` or `texts_to_matrix`.
seq = tokenizer_obj.texts_to_sequences(txt)
print(seq)
padded = pad_sequences(seq, maxlen=300)
print(padded.shape)
pred = model.predict(padded)
print(pred)
actual = np.argmax(pred) # index of maximum value
print(actual)
print(labels)


[[3793, 6000, 3128, 2191, 1115, 1, 1735, 623, 10869, 369, 854, 3128, 187, 139, 437, 489, 829, 384, 1486, 2993, 1234, 535, 75, 61, 250, 58, 698, 3793, 526, 359, 56, 221, 1115, 781, 75, 204, 18864, 854, 3128, 623, 472, 1759, 832, 7136, 10, 10, 181, 181, 2191, 3128, 4466, 2056, 399, 1751, 2056, 129, 448, 408, 10869, 3837, 189, 54, 131, 1115, 4093, 343, 225, 437, 486, 3128, 79, 363, 13, 79, 419, 486, 3, 215, 472, 1463, 593, 17, 4, 484, 489, 486, 350]]
(1, 300)
[[3.1582666e-05 6.3584810e-01 1.1838320e-02 2.1556732e-01 1.3671468e-01]]
1
['entertainment', 'tech', 'politics', 'sport', 'business']


In [ ]:
model.predict_classes(padded)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array([1])

In [ ]:
expected = y_test
y_pred = model.predict_classes(np.array(test_pad))
print(y_pred.shape)
y_pred

(223,)


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array([3, 0, 2, 3, 4, 1, 0, 1, 0, 2, 3, 2, 2, 3, 3, 3, 3, 2, 2, 0, 1, 0,
       1, 4, 0, 4, 4, 3, 2, 2, 2, 4, 0, 4, 3, 0, 3, 2, 1, 3, 1, 4, 0, 3,
       2, 4, 3, 2, 1, 3, 3, 3, 4, 4, 0, 0, 0, 2, 4, 4, 1, 0, 2, 0, 3, 0,
       4, 0, 3, 0, 0, 2, 1, 0, 1, 0, 3, 1, 2, 2, 2, 2, 1, 1, 2, 4, 1, 3,
       1, 0, 1, 4, 3, 3, 3, 1, 2, 2, 3, 0, 3, 2, 4, 3, 2, 3, 0, 4, 0, 2,
       3, 2, 0, 1, 4, 0, 4, 0, 1, 3, 4, 4, 0, 4, 2, 0, 0, 3, 1, 3, 3, 3,
       1, 0, 1, 0, 3, 3, 3, 2, 1, 1, 0, 0, 1, 2, 4, 0, 1, 3, 1, 3, 3, 0,
       4, 0, 2, 0, 1, 0, 4, 4, 4, 1, 1, 3, 4, 0, 2, 4, 3, 3, 4, 3, 1, 1,
       2, 0, 2, 2, 4, 3, 4, 0, 2, 2, 4, 0, 1, 1, 2, 0, 2, 0, 3, 3, 4, 2,
       3, 1, 0, 2, 4, 4, 3, 4, 1, 0, 4, 1, 0, 4, 4, 4, 3, 3, 2, 3, 1, 3,
       1, 4, 2])

In [ ]:
from sklearn.metrics import accuracy_score
acc = accuracy_score(y_pred , test_encode)
acc

0.9820627802690582

In [ ]:
from sklearn import metrics
print(metrics.confusion_matrix(test_encode, y_pred))

[[48  0  1  1  1]
 [ 0 39  0  0  0]
 [ 0  0 41  0  1]
 [ 0  0  0 51  0]
 [ 0  0  0  0 40]]


In [ ]:
np.unique(test_encode, return_counts=True)

(array([0, 1, 2, 3, 4]), array([51, 39, 42, 51, 40]))